In [45]:
import requests
import pymongo
from pymongo import MongoClient
from collections import OrderedDict
from pprint import pprint

In [12]:
url = "https://us-ie-big-data-technologies.ew.r.appspot.com/api/v1/events"
headers = {"accept": "application/json"} 
params = {'collection_slug': 'cryptopunks', 
            'occurred_after': '2021-09-01T00:00:00+02:00',
            'occurred_before':'2021-10-01T00:00:00+02:00',
            'offset':0,
            'limit':10000}
response = requests.request("GET",url, headers=headers, params=params)

In [13]:
api_events = response.json()['asset_events']
#api_events

In [14]:
client = MongoClient()

In [15]:
database = client['chimps']
database.drop_collection('cryptos')
collection = database['cryptos']
collection.insert_many(api_events)

In [19]:
print("Number of event transactions = ",collection.count_documents({}))

Number of event transactions =  534


In [24]:
things = {}
results = collection.find({"event_type": "successful"}
).sort('total_price', pymongo.DESCENDING).limit(10)
for result in results:
    asset = result['asset']  
    token_id = asset['token_id']
    price = result['total_price']
    things[token_id] = price
things

{'3634': '99990000000000000000',
 '8922': '99990000000000000000',
 '7633': '99980000000000000000',
 '9103': '99900000000000000000',
 '1672': '99750000000000000000',
 '3239': '99690000000000000000',
 '3533': '99500000000000000000',
 '3378': '99500000000000000000',
 '3173': '99000000000000000000',
 '4610': '99000000000000000000'}

In [26]:
len(collection.find({}).distinct('asset.name'))

395

In [41]:
pipeline =[
    {
        '$addFields': {
            'tempdate': {
                '$dateFromString': {
                    'dateString': '$transaction.timestamp'
                }
            }
        }
    }, {
        '$addFields': {
            'year': {
                '$year': '$tempdate'
            }, 
            'month': {
                '$month': '$tempdate'
            }, 
            'day': {
                '$dayOfYear': '$tempdate'
            }
        }
    }, {
        '$group': {
            '_id': {
                'year': '$year', 
                'day': '$day'
            }, 
            'count': {
                '$sum': 1
            }
        }
    }, {
        '$sort': {
            'count': -1
        }
    }
]

In [44]:
thing = collection.aggregate(pipeline)

TypeError: 'CommandCursor' object is not subscriptable

In [47]:
pprint(list(thing)) #day 259 correlates to the 16th of Sept

[{'_id': {'day': 259, 'year': 2021}, 'count': 90},
 {'_id': {'day': 273, 'year': 2021}, 'count': 44},
 {'_id': {'day': 267, 'year': 2021}, 'count': 29},
 {'_id': {'day': 244, 'year': 2021}, 'count': 27},
 {'_id': {'day': 258, 'year': 2021}, 'count': 27},
 {'_id': {'day': 268, 'year': 2021}, 'count': 26},
 {'_id': {'day': 245, 'year': 2021}, 'count': 25},
 {'_id': {'day': 272, 'year': 2021}, 'count': 21},
 {'_id': {'day': 246, 'year': 2021}, 'count': 21},
 {'_id': {'day': 266, 'year': 2021}, 'count': 19},
 {'_id': {'day': 247, 'year': 2021}, 'count': 19},
 {'_id': {'day': 250, 'year': 2021}, 'count': 18},
 {'_id': {'day': 271, 'year': 2021}, 'count': 16},
 {'_id': {'day': 249, 'year': 2021}, 'count': 14},
 {'_id': {'day': 248, 'year': 2021}, 'count': 13},
 {'_id': {'day': 264, 'year': 2021}, 'count': 12},
 {'_id': {'day': 256, 'year': 2021}, 'count': 11},
 {'_id': {'day': 269, 'year': 2021}, 'count': 11},
 {'_id': {'day': 265, 'year': 2021}, 'count': 10},
 {'_id': {'day': 255, 'year': 2